In [269]:
import pandas as pd
# ! pip install unidecode

In [270]:
bigrams = pd.read_csv("bigram_data.csv")
bigrams.head()

,custom_id,twitter_handle,bigram
0,263,RepFinkenauer,"(health,care)"
1,263,RepFinkenauer,"(care,act)"
2,263,RepFinkenauer,"(million,americans)"
3,263,RepFinkenauer,"(pre,existing)"
4,263,RepFinkenauer,"(existing,conditions)"


In [271]:
handles = pd.read_csv("Congressional Record.csv")
handles.head()

,Congress_id,Name,State,Twitter,Facebook,Position,Party
0,263,Abby Finkenauer,Iowa 1st District,@RepFinkenauer,https://www.facebook.com/RepAbbyFinkenauer,House,Democratic Party
1,516,Abigail Spanberger,Virginia 7th District,@RepSpanberger,https://www.facebook.com/RepAbigailSpanberger,House,Democratic Party
2,242,Adam Kinzinger,Illinois 16th District,@RepKinzinger,https://www.facebook.com/RepKinzinger,House,Republican Party
3,142,Adam Schiff,California 28th District,@RepAdamSchiff,https://www.facebook.com/RepAdamSchiff,House,Democratic Party
4,528,Adam Smith,Washington 9th District,@RepAdamSmith,https://www.facebook.com/RepAdamSmith,House,Democratic Party


In [272]:
scores = pd.read_csv("trump_score_latest.csv")
scores.head()

,congress,chamber,bioguide,last_name,state,district,party,votes,agree_pct,predicted_agree,net_trump_vote
0,0,house,A000055,Aderholt,AL,4.0,Republican,122,0.975410,0.956077,63.0
1,115,house,A000055,Aderholt,AL,4.0,R,95,0.968421,0.946349,63.0
2,116,house,A000055,Aderholt,AL,4.0,R,27,1.000000,0.990306,63.0
3,0,house,A000367,Amash,MI,3.0,Republican,120,0.616667,0.792576,9.4
4,115,house,A000367,Amash,MI,3.0,R,96,0.541667,0.847386,9.4


In [273]:
def separate_by_id(bigrams_df):
    df_l = []
    for cid in bigrams["custom_id"].unique():
        df = bigrams[bigrams["custom_id"]==cid]
        df_l.append(df)
    return df_l

In [274]:
# add Trump Score to handles

no_match = []
one_match = []
many_match = []
for index in scores.index:
    row = scores.loc[index]
    matches = handles[handles["Name"].str.contains(row["last_name"])]
    party = row["party"][0]
    chamber = row["chamber"][0].upper()
    state = row["state"][0]
    matches = matches[(matches["Party"].str.startswith(party))
       & (matches["Position"].str.startswith(chamber))
       & (matches["State"].str.startswith(state))]
    if (len(matches) == 0) and row["congress"] == 116:
        no_match.append(row)
    elif len(matches) == 1:
        handles.at[matches.index[0], "score"] = row["agree_pct"]
        one_match.append((row, matches))
    elif len(matches) > 1:
        many_match.append((row, matches))
        
        
print("No Matches:\t", len(no_match))
print("One Match:\t", len(one_match))
print("Many Matches:\t", len(many_match))
    
        

No Matches:	 6
One Match:	 1485
Many Matches:	 12


In [257]:
df_l = separate_by_id(bigrams)

In [259]:
def bigram_freq(df_list: [pd.DataFrame], handles: [pd.DataFrame],
                bigrams: [str]):
    dict_list = []
    for df in df_list:
        v = df["bigram"].value_counts()
        d = {"custom_id": df["custom_id"].iloc[0]}
        try:
            handle = handles[handles["Congress_id"]==d["custom_id"]]
            d["name"] = handle["Name"].item().strip()
            d["state"] = handle["State"].item().strip()
            d["position"] = handle["Position"].item().strip()[0]
            d["party"] = handle["Party"].item().strip()[0]
        except KeyError as e:
            pass
        for bigram in bigrams:
            d[bigram] = 0
            if bigram in v:
                d[bigram] = v[bigram]
        dict_list.append(d)
    return dict_list

In [260]:
dl = bigram_freq(df_l, handles, bigrams["bigram"].value_counts().index)

In [ ]:
bigram_count = pd.DataFrame(dl).set_index("custom_id")
bigram_count.head()

In [267]:
# for row in no_match:
#     display(row)
# no_match[0]["last_name"]
# handles[handles["Name"].str.contains("Barragan")]
handles

,Congress_id,Name,State,Twitter,Facebook,Position,Party,score
0,263,Abby Finkenauer,Iowa 1st District,@RepFinkenauer,https://www.facebook.com/RepAbbyFinkenauer,House,Democratic Party,0.000000
1,516,Abigail Spanberger,Virginia 7th District,@RepSpanberger,https://www.facebook.com/RepAbigailSpanberger,House,Democratic Party,0.000000
2,242,Adam Kinzinger,Illinois 16th District,@RepKinzinger,https://www.facebook.com/RepKinzinger,House,Republican Party,0.730769
3,142,Adam Schiff,California 28th District,@RepAdamSchiff,https://www.facebook.com/RepAdamSchiff,House,Democratic Party,0.000000
4,528,Adam Smith,Washington 9th District,@RepAdamSmith,https://www.facebook.com/RepAdamSmith,House,Democratic Party,0.000000
5,339,Adrian Smith,Nebraska 3rd District,@RepAdrianSmith,https://www.facebook.com/AdrianSmithNE,House,Republican Party,NaN
6,364,Adriano Espaillat,New York 13th District,@EspaillatNY,https://www.facebook.com/EspaillatForCongress,House,Democratic Party,0.000000
7,501,Al Green,Texas 9th District,@RepAlGreen,https://www.facebook.com/Congressman-Al-Green-...,House,Democratic Party,0.000000
8,212,Al Lawson,Florida 5th District,@AlLawsonJr,https://www.facebook.com/AlLawsonJr,House,Democratic Party,0.000000
9,163,Alan Lowenthal,California 47th District,@RepLowenthal,https://www.facebook.com/RepLowenthal,House,Democratic Party,0.000000


In [265]:
scores[scores["last_name"]=="Finkenauer"]

,congress,chamber,bioguide,last_name,state,district,party,votes,agree_pct,predicted_agree,net_trump_vote
433,0,house,F000467,Finkenauer,IA,1.0,Democrat,27,0.0,0.28838,3.5
434,116,house,F000467,Finkenauer,IA,1.0,D,27,0.0,0.28838,3.5


In [158]:
row["last_name"]

'Aderholt'

In [171]:
for index in scores.index[:5]:
    row = scores.loc[index]
    print(row["last_name"])
    print(type(row["last_name"]))
    handles[handles["Name"].str.contains(row["last_name"])]

Aderholt
<class 'str'>
Aderholt
<class 'str'>
Aderholt
<class 'str'>
Amash
<class 'str'>
Amash
<class 'str'>


In [166]:
row

congress                    0
chamber                 house
bioguide              A000055
last_name            Aderholt
state                      AL
district                    4
party              Republican
votes                     122
agree_pct             0.97541
predicted_agree      0.956077
net_trump_vote             63
Name: 0, dtype: object

In [170]:
handles[handles["Name"].str.contains("Aderholt")]

,Congress_id,Name,State,Twitter,Facebook,Position,Party
345,104,Robert Aderholt,Alabama 4th District,@Robert_Aderholt,https://www.facebook.com/RobertAderholt,House,Republican Party


In [185]:
matches[(matches["State"].str.startswith("A"))]

,Congress_id,Name,State,Twitter,Facebook,Position,Party
115,108,Don Young,Alaska At-Large,@repdonyoung,https://www.facebook.com/RepDonYoung,House,Republican Party


In [192]:
matches[(matches["Party"].str.startswith("R"))
       & (matches["Position"].str.startswith("h".upper()))
       & (matches["State"].str.startswith("A"))]

,Congress_id,Name,State,Twitter,Facebook,Position,Party
115,108,Don Young,Alaska At-Large,@repdonyoung,https://www.facebook.com/RepDonYoung,House,Republican Party


In [183]:
matches["State"].str.startswith("A")

115     True
537    False
Name: State, dtype: bool